## Import Modules and Dependencies

In [6]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers

## Load the Dataset

In [51]:
dataset = '/usercode/Roadsign_dataset'
tf_dataset = tf.keras.preprocessing.image_dataset_from_directory(dataset, seed=41, shuffle=True, image_size=(256,256), batch_size=32)

NotFoundError: Could not find directory /usercode/Roadsign_dataset

## Get the Names and Numbers of Classes

In [ ]:
class_names = tf_dataset.class_names
number_of_classes = len(class_names)
print(f'Names of classes are {class_names} and there are {number_of_classes} in the dataset')

## Visualize the Dataset

In [ ]:
plt.figure(figsize=(16,16))

for image_batch, labels_batch in tf_dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis('off')

## Split the Dataset

In [ ]:
def tf_split_data(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=10)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_set = ds.take(train_size)
    val_set = ds.skip(train_size).take(val_size)
    test_set = ds.skip(train_size).skip(val_size)

    return train_set, val_set, test_set

train,val,test = tf_split_data(tf_dataset)

## Declare Constants

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 3

## Shuffling Data and Prefetching

In [ ]:
train = train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val = val.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
test = test.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)

## Resizing and Rescaling Layer

In [ ]:
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1./255)
])

## Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

## Implementing Data Augmentation

In [ ]:
train.map(
    lambda x,y: (data_augmentation(x,training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

## Define the Parameters for Model Building

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = number_of_classes #7

## Architecturing Convolution Neural Network

In [ ]:
model = models.Sequential([
    resize_rescale,
    layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.build(input_shape=input_shape)

## Model Compilation

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

## Train the Model

In [ ]:
history = model.fit(
    train, 
    batch_size = BATCH_SIZE,
    validation_data = val,
    verbose=1,
    epochs=EPOCHS
)

## Evaluate the Model

In [ ]:
scores = model.evaluate(test)

## Defining Training and Validation Metrics

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

## Plotting Accuracy and Loss Curves

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label='Training Accuracy', color='black')
plt.plot(range(EPOCHS),val_acc,label='Validation Accuracy', color='red')
plt.legend()
plt.title("Training/Validation Accuracy vs Epochs", fontsize=12)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1,2,2)
plt.plot(range(EPOCHS), loss, label='Training Loss', color='black')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss', color='red')
plt.legend()
plt.title('Training/Validation Loss vs Epochs', fontsize=12)
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

## Predict function

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img.numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

## Test the Model on a Single Image

In [ ]:
plt.figure(figsize=(8,8))
plt.axis('off')
for images_batch,labels_batch in test.take(1):

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print('First image to predict:')
    plt.imshow(first_image)
    print('Actual label: ', class_names[first_label])

    batch_prediction = model.predict(images_batch)
    print('Predicted label: ', class_names[np.argmax(batch_prediction[0])])


    

## Test the Model on Multiple Images

In [ ]:
plt.figure(figsize=(15,15))

for image_batch, label_batch in test.take(1):
    for i in range(12):
        ax = plt.subplot(4,3,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        predicted_class,confidence = predict(model, image_batch[i])
        actual_class = class_names[label_batch[i]]

        plt.title(f'Actual: {actual_class}, \n Predicted: {predicted_class}')
        plt.axis('off')
